# The Objective of this project is to Segmenting Customers Based on purchase Behaviour

## Step 1: Load the Data

In [3]:
import os
os.chdir(r'C:\Users\Lenovo\Documents\Business Performance KPI Dashboard')
print(os.listdir())

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'C:\\Users\\Lenovo\\Documents\\Business Performance KPI Dashboard'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import openpyxl

In [ ]:
# Changed encoding to 'ISO-8859-1' (ANSI) because the file does not support UTF-8 and requires compatibility.
df = pd.read_csv('Online_Retail.csv', encoding='ISO-8859-1')

In [ ]:
# Let's peak into the data
df

In [ ]:
# Basic information
print(df.info())

## Step 2: Data Cleaning

In [ ]:
# Check for missing values
print(df.isnull().sum())

In [ ]:
# Let's check how the severe missing values is in percentage
missing_percent = df.isnull().sum() / len(df) * 100
print(missing_percent)


In [ ]:
missing_customerid = df[df['CustomerID'].isnull()]
print(missing_customerid)

In [ ]:
# First, fill the missing CustomerID with 'Unknown'
df['CustomerID'] = df['CustomerID'].fillna('Unknown')

# Create the flag column: 1 for known customers, 0 for unknown customers
df['CustomerID_flag'] = (df['CustomerID'] != 'Unknown').astype(int)

# Check the result
print(df)

In [ ]:
# Handling Decription missing values. Many data is also missing in such rows.
df = df.dropna(subset=['Description'])

In [ ]:
df.isnull().sum()

In [ ]:
# Check for Duplicate entries
print(df.duplicated().sum())

In [ ]:
# Display raws that are duplicates
duplicate_raws = df[df.duplicated()]
print(duplicate_raws)

In [ ]:
import re
# Strip whitespace from all string columns
df = df.map(lambda x: x.strip() if isinstance(x, str) else x)

# Remove non-printable characters from all string columns
df = df.map(lambda x: re.sub(r'\s+', ' ', x) if isinstance(x, str) else x)
# Convert all string columns to lowercase
df['Description'] = df['Description'].str.lower()
df['Country'] = df['Country'].str.lower()

# Check for duplicates across all columns
df.duplicated().sum()


## Duplicate Check Explanation

In reviewing the dataset, we performed a check for duplicate entries based on the following key columns:

- **InvoiceNo**: Each invoice number is typically unique to a transaction. However, when multiple items are purchased under the same invoice, there will be multiple entries with the same **InvoiceNo**.

- **Quantity**: The quantity of items purchased can vary for each entry within the same invoice, resulting in separate rows for different items.

- **InvoiceDate**: The invoice date is generally the same for all items in a single transaction, but slight variations in the exact time recorded might cause multiple rows to appear.

- **CustomerID**: The **CustomerID** corresponds to the customer making the purchase. Since a customer may buy multiple items under the same invoice, this will appear as repeated rows for the same transaction.

- **Country**: The **Country** indicates the location of the customer. Multiple items from the same invoice may share the same country, which results in multiple rows under the same country for the same customer.

### Conclusion:

After investigating, we confirm that the dataset does not contain true duplicate rows. The system marks multiple entries as duplicates because these columns (InvoiceNo, Quantity, InvoiceDate, CustomerID, and Country) may repeat for the same transaction, where different items are bought under the same invoice.

These entries are not actual duplicates but represent different items purchased under the same invoice. If necessary, we can remove entries considered duplicates based on these columns or further refine the analysis criteria.



In [ ]:
# Let's find out invalid or negative quantities.
print((df['Quantity'] <= 0).sum())

In [ ]:
negative_quantity = df[df['Quantity'] <=0]
print(negative_quantity)

In [ ]:
# Flag negative quantities for potential future analysis, allowing for further exploration if needed.
df['Negative_Quantity_Flag'] = df['Quantity'].apply(lambda x: 1 if x < 0 else 0)

In [ ]:
# Count rows where unitprice is zero
zero_unitprice_count = (df['UnitPrice'] == 0).sum()

print(zero_unitprice_count)


In [ ]:
# Filter rows where unitprice is zero
rows_with_zero_unitprice = df[df['UnitPrice'] == 0]

# Display rows with zero unitprice and their descriptions
print(rows_with_zero_unitprice[['Description', 'UnitPrice']])

In [ ]:
# Replace 0 values in Quantity with NaN
df['UnitPrice'] = df['UnitPrice'].replace(0, np.nan)

# Check the result
print(df[df['UnitPrice'].isna()].head())

In [ ]:
# Let's do the Imputation
# Impute missing UnitPrice by StockCode (mean of UnitPrice by StockCode)
df['UnitPrice'] = df.groupby('StockCode')['UnitPrice'].transform(lambda x: x.fillna(x.mean()))

# Impute remaining missing UnitPrice by Description (mean of UnitPrice by Description)
df['UnitPrice'] = df.groupby('Description')['UnitPrice'].transform(lambda x: x.fillna(x.mean()))

# If any values are still missing, impute with the global mean of UnitPrice
df['UnitPrice'] = df['UnitPrice'].fillna(df['UnitPrice'].mean())

# Verifying if any missing values remain
print(df[df['UnitPrice'].isna()])

In [ ]:
# let's check further.
print(df[df['UnitPrice'].isna() | (df['UnitPrice'] == 0)])

In [ ]:
# Check the summary statistics of UnitPrice
print(df['UnitPrice'].describe())

# Check the first few rows to ensure imputation worked correctly
print(df.head())

In [ ]:
# Check rows with extreme UnitPrice values
extreme_values = df[df['UnitPrice'] < 0 | (df['UnitPrice'] > 1000)]
print(extreme_values)

In [ ]:
df = df[df['UnitPrice'] > 0]

In [ ]:
# Remove rows where Description contains "adjust bad debt"
df = df[~df['Description'].str.contains('adjust bad debt', case=False, na=False)]

In [ ]:
# Check rows with extreme UnitPrice values
extreme_values = df[df['UnitPrice'] < 0 | (df['UnitPrice'] > 1000)]
print(extreme_values)

In [ ]:
df

In [ ]:
# Create Total Price column
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

print(df)

In [ ]:
df.to_csv(r'C:\Users\Lenovo\Downloads\segmenting_customers.csv', index=False)

In [ ]:
print("Cleaned Data:")